### Logistic Regression for Classification

### Import Libraries

In [76]:
import numpy as np
import pandas as pd

### Load Dataset

In [77]:
"""Load data here"""
data = pd.read_csv('data.csv')
data.head()

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,23.720338,89.592641,7.335604,50.501694,1032.378759,rain
1,27.879734,46.489704,5.952484,4.990053,992.614190,no rain
2,25.069084,83.072843,1.371992,14.855784,1007.231620,no rain
3,23.622080,74.367758,7.050551,67.255282,982.632013,rain
4,20.591370,96.858822,4.643921,47.676444,980.825142,no rain


### Check for Missing Values

In [78]:
"""Check for Missing Data"""
print(data.isnull().sum())

Temperature    0
Humidity       0
Wind_Speed     0
Cloud_Cover    0
Pressure       0
Rain           0
dtype: int64


### Perform Minmax Scaling

In [79]:
def minmax_scaling (data, column):
    min = data[column].min()
    max = data[column].max()
    return (data[column] - min) / (max - min)

In [80]:
"""Perform Minmax Scaling on Appropraite Columns"""
data['Temperature'] = minmax_scaling(data, 'Temperature')
data['Humidity'] = minmax_scaling(data, 'Humidity')
data['Wind_Speed'] = minmax_scaling(data, 'Wind_Speed')
data['Cloud_Cover'] = minmax_scaling(data, 'Cloud_Cover')
data['Pressure'] = minmax_scaling(data, 'Pressure')
data

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,0.548885,0.851343,0.366485,0.504954,0.748370,rain
1,0.715305,0.235520,0.297292,0.049759,0.180070,no rain
2,0.602850,0.758193,0.068145,0.148433,0.388977,no rain
3,0.544954,0.633821,0.352225,0.672518,0.037409,rain
4,0.423693,0.955157,0.231829,0.476696,0.011586,no rain
...,...,...,...,...,...,...
2495,0.471715,0.218107,0.590184,0.550391,0.538389,no rain
2496,0.702452,0.235407,0.544045,0.397069,0.408404,no rain
2497,0.724449,0.197337,0.144443,0.758410,0.273037,no rain
2498,0.191548,0.398658,0.118308,0.023641,0.950212,no rain


### Perform Encoding

In [81]:
"""Encode target column which is a categorical attribute (Hint: data[col].map)"""
data['Rain'] = data['Rain'].map({'rain': 1, 'no rain': 0})
data

,Temperature,Humidity,Wind_Speed,Cloud_Cover,Pressure,Rain
0,0.548885,0.851343,0.366485,0.504954,0.748370,1
1,0.715305,0.235520,0.297292,0.049759,0.180070,0
2,0.602850,0.758193,0.068145,0.148433,0.388977,0
3,0.544954,0.633821,0.352225,0.672518,0.037409,1
4,0.423693,0.955157,0.231829,0.476696,0.011586,0
...,...,...,...,...,...,...
2495,0.471715,0.218107,0.590184,0.550391,0.538389,0
2496,0.702452,0.235407,0.544045,0.397069,0.408404,0
2497,0.724449,0.197337,0.144443,0.758410,0.273037,0
2498,0.191548,0.398658,0.118308,0.023641,0.950212,0


### Divide Data into Training and Testing

In [82]:
def train_test_split (data, ratio):
    indices = np.random.permutation(data.shape[0])
    test_set_size = int(data.shape[0] * ratio)
    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]

    return data.iloc[train_indices], data.iloc[test_indices]

In [83]:
"""Understand the above function and divide data into X_train, X_test, y_train, y_test"""
train_data, test_data = train_test_split(data, 0.3) # 70% train , 30% test
X_train = train_data[['Temperature','Humidity','Wind_Speed','Cloud_Cover','Pressure']].values
y_train = train_data[['Rain']].values.reshape(-1, 1)
X_test = test_data[['Temperature','Humidity','Wind_Speed','Cloud_Cover','Pressure']].values
y_test = test_data[['Rain']].values.reshape(-1, 1)

### Compute the Sigmoid Function

In [84]:
"""This function is used by consequent functions"""

def sigmoid (z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
    
    Hint: np.exp()
    """
    return 1 / (1 + np.exp(-z))   # 1/ 1 + e^-z

In [85]:
"""Test above function"""

value = 0
print (f"sigmoid({value}) = {sigmoid(value)}")

sigmoid(0) = 0.5


### Compute the Cost Function

In [86]:
"""This function is used by consequent functions"""
#measure how far from the y truth

def compute_cost(X, y, w): 
    """
    Computes the logistic regression cost using a loop.

    Args:
      X : (ndarray Shape (m,n)) training data
      y : (ndarray Shape (m,1))  target values
      w : (ndarray Shape (n,1))  weights

    Returns:
      cost : scalar
    """
    m = X.shape[0]
    total_cost = 0

    for i in range(m):
        z = X @ w          
        h = sigmoid(z)
        cost_i = - y[i] * np.log(h) - (1 -  y[i]) * np.log(1 - h)
        total_cost += cost_i

    return total_cost / m

In [87]:
"""Test above function"""
m, n = X_train.shape
initial_w = np.zeros(n)    #base line to start computing
cost = compute_cost(X_train, y_train, initial_w)
print('Cost at initial w: {}'.format(cost))

Cost at initial w: [0.69314718 0.69314718 0.69314718 ... 0.69314718 0.69314718 0.69314718]


### Compute Gradient of the Cost Function

In [88]:
"""This function is used by consequent function"""

def compute_gradient(X, y, w): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model                  
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w.     
    """
    m = X.shape[0]
    z = np.dot(X, w)
    h = sigmoid(z)
    gradient = (1/m) * np.dot(X.T, (h - y))    #(1/m)* ((h-y) x)
    return gradient

In [89]:
"""Test above function"""
# Compute and display gradient with w and b initialized to zeros
initial_w = np.zeros(X_train.shape[1]).reshape(-1, 1)
dj_dw = compute_gradient(X_train, y_train, initial_w)
print(f'dj_dw at initial w: {dj_dw.tolist()}' ) # numpy array into python list

dj_dw at initial w: [[0.21737846198784957], [0.156334634011174], [0.19030023474682486], [0.1618833426733165], [0.185922916857178]]


### Calcualte Weights Using Gradient Descent

In [90]:
def gradient_descent(X, y, w_in, cost_function, gradient_function, alpha, num_iters): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) training data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model      
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters/weights of the model after
          running gradient descent      
    """
    w = w_in.copy()
    for i in range(num_iters):
        grad = gradient_function(X, y, w)
        w -= alpha * grad                           #theta(j)-alpha *grad
        if i % 100 == 0:
            cost = cost_function(X, y, w)
            print(f"Iteration {i}: Cost {cost}")
    return w


In [91]:
"""Call above function on your data with appropraite parameters and fetch the optimal weights."""
initial_w = np.zeros(X_train.shape[1]) # NO OF COLS


alpha = 0.5      
num_iters = 1000  
y_train = y_train.flatten()
optimal_w = gradient_descent(X_train, y_train, initial_w, compute_cost, compute_gradient, alpha, num_iters)

print("Optimal weights:", optimal_w)

Iteration 0: Cost [0.6251264  0.59773519 0.59609349 ... 0.61471764 0.61740185 0.65283892]
Iteration 100: Cost [0.37212557 0.36875125 0.37444612 ... 0.37559813 0.36127879 0.54707788]
Iteration 200: Cost [0.3676153  0.38600471 0.39904365 ... 0.38558058 0.35861857 0.62965382]
Iteration 300: Cost [0.36389216 0.40418021 0.42438194 ... 0.3906455  0.35772515 0.69036605]
Iteration 400: Cost [0.36141947 0.42074015 0.44725379 ... 0.39325911 0.35809807 0.73637138]
Iteration 500: Cost [0.35986352 0.43514357 0.46704865 ... 0.39465724 0.359148   0.77232335]
Iteration 600: Cost [0.35890529 0.44749198 0.48395878 ... 0.39541611 0.36050902 0.80109482]
Iteration 700: Cost [0.35832809 0.45804519 0.49836497 ... 0.39582062 0.36197604 0.82453688]
Iteration 800: Cost [0.35799367 0.46707299 0.5106514  ... 0.39602111 0.36343743 0.84390051]
Iteration 900: Cost [0.3578144  0.47481456 0.521156   ... 0.39610143 0.36483412 0.86006694]
Optimal weights: [-5.32147613  1.9894481  -2.12602365  1.96890252 -1.9024293 ]


### Calculate Predictions on Test Test

In [94]:
def predict(X, w): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w.
    
    Args:
      X : (ndarray Shape (m,n)) test data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model            

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    
    probs = sigmoid(np.dot(X, w))
    return (probs >= 0.5).astype(int)


### Calculate Accuracy of Model

In [96]:
"""Build a logic to estimate the model's accuracy"""
y_pred = predict(X_test, optimal_w )
accuracy = np.mean(y_pred == y_test)
print(f"Model accuracy on test set: {accuracy*100:.3f}")

Model accuracy on test set: 76.838
